In [ ]:
# References: 
# Original Plotting Code: https://github.com/abhijeet3922/FaceEmotion_ID/blob/master/real_time_video.py
# Frontal Detector: https://github.com/opencv/opencv/tree/master/data/haarcascades

from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np

In [ ]:
# set path for frontal detector and emotion detector
detection_model_path = 'haarcascade/haarcascade_frontalface_default.xml'
emotion_model_path = 'final_cnn_model.h5'

In [ ]:
# load frontal detector model and our groups trained emotion detector
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)

# emotion labels
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
# start video stream
cv2.namedWindow('Facial Expression Recognizer')
camera = cv2.VideoCapture(0)

while camera.isOpened():
    #read the frame
    frame = camera.read()[1]
    frame = imutils.resize(frame,width=1000)
    
    # get grayscale of frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect faces in grayscale image
    faces = face_detection.detectMultiScale(gray, scaleFactor=1.1, 
                                            minNeighbors=5, minSize=(30,30),
                                            flags=cv2.CASCADE_SCALE_IMAGE)
    
    if len(faces) > 0:
        faces = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
        
        # find region of interest in the grayscale image and then prepare
        roi = gray[fY:fY + fH, fX:fX + fW]
        
        # resize image to match FER2013 size (48x48 pixels) 
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # make prediction on region of interest
        preds = emotion_classifier.predict(roi)[0]
        
        # final prediction and label
        emotion_probability = np.max(preds)
        label = emotions[preds.argmax()]

    frameClone = frame.copy()    
    canvas = np.zeros((250, 300, 3), dtype="uint8")   
    
    for (i, (emotion, prob)) in enumerate(zip(emotions, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5),
                (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (255, 255, 255), 2)
                cv2.putText(frameClone, label, (fX, fY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                              (0, 0, 255), 2)

    cv2.imshow('Facial Expression Recognizer', frameClone)
    cv2.imshow('Emotion Probabilities', canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
camera.release()
cv2.destroyAllWindows()